In [ ]:
import fsspec
import xarray as xr
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import pytorch_lightning as pl
import torch.nn as nn
import torch

import pyqg_explorer.dataset.forcing_dataset as forcing_dataset
import pyqg_explorer.models.base_model as base_model

In [ ]:
data_full=xr.open_zarr(fsspec.get_mapper(f'/scratch/zanna/data/pyqg/publication/eddy/forcing1.zarr'), consolidated=True)
data_forcing=data_full.q_subgrid_forcing.isel(lev=1)
data_forcing=data_forcing.stack(snapshot=("run","time"))
data_forcing=data_forcing.transpose("snapshot","y","x")
data_forcing.isel(snapshot=54).plot()

In [ ]:
data_q=data_full.q.isel(lev=1)
data_q=data_q.stack(snapshot=("run","time"))
data_q=data_q.transpose("snapshot","y","x")
data_q.isel(snapshot=54).plot()

In [ ]:
dataset=forcing_dataset.ForcingDataset(data_q,data_forcing)

In [ ]:
train_loader = DataLoader(
    dataset,
    batch_size=32,
    sampler=SubsetRandomSampler(dataset.train_idx),
)
valid_loader = DataLoader(
    dataset,
    batch_size=32,
    sampler=SubsetRandomSampler(dataset.valid_idx),
)

In [ ]:
data=next(iter(train_loader))

In [ ]:
data[0].shape

In [ ]:
model=base_model.AndrewCNN(1,1)

In [ ]:
model(data[0]).shape